In [ ]:


mport pandas as pd
import os
import sys
import re
from google.colab import files
import io

class DataPreprocessor:
    def __init__(self, df):
        self.df = df.copy()

    def clean_data(self):
        print("Starting data preprocessing...")
        initial_rows = len(self.df)

        print("Processing Alt Text...")
        self._process_alt_text()

        print("Removing self-referential links...")
        self._remove_self_links()

        print("Removing paginated URLs...")
        self._remove_paginated_urls()

        print("Filtering by type and removing type columni...")
        self._filter_by_type()

        print("Removing unwanted anchor texts...")
        self._remove_unwanted_anchors()

        print("Keeping only required columns...")
        self._keep_required_columns()

        final_rows = len(self.df)
        print(f"Preprocessing complete. Rows reduced from {initial_rows} to {final_rows}")

        return self.df

    def _process_alt_text(self):
        if 'Alt Text' in self.df.columns:
            self.df['Anchor'] = self.df.apply(
                lambda row: row['Alt Text'] if pd.isna(row['Anchor']) and not pd.isna(row['Alt Text'])
                else row['Anchor'],
                axis=1
            )

    def _remove_self_links(self):
        self.df['Source_normalized'] = self.df['Source'].str.split('#').str[0]
        self.df['Destination_normalized'] = self.df['Destination'].str.split('#').str[0]
        self.df = self.df[self.df['Source_normalized'] != self.df['Destination_normalized']]
        self.df = self.df.drop(['Source_normalized', 'Destination_normalized'], axis=1)

    def _remove_paginated_urls(self):
        pagination_patterns = ['\/page\/', '\?p=', 'page=']
        pattern = '|'.join(pagination_patterns)

        mask = ~(
            self.df['Source'].str.contains(pattern, regex=True, na=False) |
            self.df['Destination'].str.contains(pattern, regex=True, na=False)
        )
        self.df = self.df[mask]

    def _filter_by_type(self):
        if 'Type' in self.df.columns:
            self.df = self.df[self.df['Type'].str.lower() == 'hyperlink']
            self.df = self.df.drop('Type', axis=1)

    def _remove_unwanted_anchors(self):
        unwanted_terms = ['login', 'log in', 'register', 'click here', 'next', 'previous']
        pattern = '|'.join(map(re.escape, unwanted_terms))
        self.df = self.df[~self.df['Anchor'].str.lower().str.contains(pattern, regex=True, na=False)]

    def _keep_required_columns(self):
        required_columns = ['Source', 'Destination', 'Anchor']
        self.df = self.df[required_columns]

def analyze_multiple_destinations(df):
    """Identify all anchor texts that link to multiple destinations"""
    multiple_dest_data = []

    # Group by anchor text
    anchor_groups = df.groupby('Anchor')

    for anchor, group in anchor_groups:
        # Get unique destinations
        destinations = group['Destination'].unique()
        if len(destinations) > 1:
            # Count occurrences of each destination
            dest_counts = group['Destination'].value_counts()
            total_occurrences = len(group)

            # Add a row for each destination
            for dest, count in dest_counts.items():
                multiple_dest_data.append({
                    'anchor_text': anchor,
                    'destination_url': dest,
                    'occurrence_count': count,
                    'total_occurrences': total_occurrences,
                    'percentage': f"{(count/total_occurrences)*100:.2f}%",
                    'unique_destinations': len(destinations),
                    'example_source': group['Source'].iloc[0]
                })

    return pd.DataFrame(multiple_dest_data)

def analyze_inconsistencies_by_most_links(df):
    """Identify inconsistencies based on most frequently used URLs"""
    inconsistencies = []

    # Group by anchor text
    anchor_groups = df.groupby('Anchor')

    for anchor, group in anchor_groups:
        # Get destination counts
        dest_counts = group['Destination'].value_counts()
        if len(dest_counts) > 1:
            # Get the most common destination
            most_common_dest = dest_counts.index[0]
            most_common_count = dest_counts.iloc[0]

            # Find all links that don't point to the most common destination
            incorrect_links = group[group['Destination'] != most_common_dest]

            for _, row in incorrect_links.iterrows():
                inconsistencies.append({
                    'source_page': row['Source'],
                    'current_destination': row['Destination'],
                    'recommended_destination': most_common_dest,
                    'anchor_text': anchor,
                    'link_count_correct': most_common_count,
                    'total_occurrences': len(group),
                    'reason': f"'{anchor}' most commonly links to {most_common_dest} ({most_common_count} times)"
                })

    return pd.DataFrame(inconsistencies)

def analyze_inconsistencies_by_ranking(df, ranking_df):
    """Identify inconsistencies based on ranking positions"""
    anomalies = []

    df['anchor_normalized'] = df['Anchor'].fillna('').str.strip().str.lower()
    ranking_df['keyword_normalized'] = ranking_df['Keyword'].fillna('').str.strip().str.lower()

    anchor_groups = df.groupby('anchor_normalized')

    for anchor, group in anchor_groups:
        if not anchor:  # Skip empty anchor texts
            continue

        ranking_data = ranking_df[ranking_df['keyword_normalized'] == anchor]

        if not ranking_data.empty:
            best_ranking = ranking_data.sort_values('Ranking Position').iloc[0]
            best_ranking_url = str(best_ranking['Top Ranking URL']).rstrip('/')
            best_ranking_position = best_ranking['Ranking Position']

            # Find all links that don't point to the best ranking URL
            incorrect_links = group[group['Destination'].str.rstrip('/') != best_ranking_url]

            for _, row in incorrect_links.iterrows():
                anomalies.append({
                    'source_page': row['Source'],
                    'current_destination': row['Destination'],
                    'recommended_destination': best_ranking_url,
                    'anchor_text': row['Anchor'],
                    'ranking_position': best_ranking_position,
                    'reason': f"'{row['Anchor']}' should link to the highest ranking page (position {best_ranking_position})"
                })

    return pd.DataFrame(anomalies)

def run_analysis():
    print("Internal Link Consistency Analyzer")
    print("=================================")

    try:
        # Upload crawl data
        print("1. Please upload your crawl data CSV file.")
        print("Required columns: 'Source', 'Destination', 'Anchor', 'Type', 'Alt Text'")
        uploaded_crawl = files.upload()
        crawl_file = list(uploaded_crawl.keys())[0]

        # Upload ranking data
        print("2. Please upload your ranking data CSV file.")
        print("Required columns: 'Keyword', 'Ranking Position', 'Top Ranking URL'")
        uploaded_ranking = files.upload()
        ranking_file = list(uploaded_ranking.keys())[0]

        # Load and preprocess crawl data
        print("Loading and preprocessing crawl data...")
        raw_df = pd.read_csv(crawl_file)
        preprocessor = DataPreprocessor(raw_df)
        cleaned_df = preprocessor.clean_data()

        # Load ranking data
        print("Loading ranking data...")
        ranking_df = pd.read_csv(ranking_file)

        # Generate all three reports
        print("Generating reports...")

        # 1. Multiple Destinations Report
        print("1. Analyzing multiple destinations...")
        multiple_dest_df = analyze_multiple_destinations(cleaned_df)
        multiple_dest_df.to_csv('multiple_destinations.csv', index=False)
        print(f"Found {len(multiple_dest_df)} instances of multiple destinations")

        # 2. Inconsistencies by Most Links Report
        print("2. Analyzing inconsistencies by link count...")
        inconsistencies_links_df = analyze_inconsistencies_by_most_links(cleaned_df)
        inconsistencies_links_df.to_csv('inconsistencies_by_most_links.csv', index=False)
        print(f"Found {len(inconsistencies_links_df)} inconsistencies based on link count")

        # 3. Inconsistencies by Ranking Report
        print("3. Analyzing inconsistencies by ranking...")
        inconsistencies_ranking_df = analyze_inconsistencies_by_ranking(cleaned_df, ranking_df)
        inconsistencies_ranking_df.to_csv('inconsistencies_by_ranking.csv', index=False)
        print(f"Foundtiple_dest_df.head())

        print("\nSample of Inconsistencies by Most Links Report:")
        display(inconsistencies_links_df.head())

        print("\nSample of Inconsistencies by Ranking Report:")
        display(inconsistencies_ranking_df.head())

        # Download reports
        print("\nDownloading reports...")
        files.download('multiple_destinations.csv')
        files.download('inconsistencies_by_most_links.csv')
        files.download('inconsistencies_by_ranking.csv')

        print("\nAnalysis complete!")

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        print("Full error details:")
        print(traceback.format_exc())
        return

# Run the analysis
run_analysis() {len(inconsistencies_ranking_df)} inconsistencies based on rankings")

        # Show samples of each report
        print("\nSample of Multiple Destinations Report:")
        display(mul